In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [ ]:
train_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape, test_df.shape

# Let's look through data

In [ ]:
train_df['SalePrice'].describe()

In [ ]:
sns.distplot(train_df['SalePrice'])

Separate out 'SalePrice' column as labels and concat 2 dataframes and preprocess together.

In [ ]:
labels = train_df['SalePrice']
labels

In [ ]:
train_df.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
train_idx = train_df.shape[0]
train_idx

In [ ]:
new_df = pd.concat([train_df, test_df], ignore_index=True, axis=0)

In [ ]:
new_df.head()

In [ ]:
new_df.shape

   # Let's check out the NaN values from each column

In [ ]:
nan_pct = [new_df[col].isna().sum() * 100/new_df.shape[0] for col in new_df.columns]
nan_df = pd.DataFrame(columns=['column', 'percentage'])
nan_df['column'] = new_df.columns
nan_df['percentage'] = nan_pct
nan_df[nan_df['percentage'] > 50]

Let's check what values are available in these columns

In [ ]:
new_df['Alley'].unique(), new_df['PoolQC'].unique(), new_df['Fence'].unique(), new_df['MiscFeature'].unique()

Let's drop these columns

In [ ]:
new_df.drop(nan_df[nan_df['percentage'] > 50]['column'].unique(), axis=1, inplace=True)

In [ ]:
nan_df[(nan_df['percentage'] < 50) & (nan_df['percentage'] > 5)]

In [ ]:
new_df['FireplaceQu'].unique()

In [ ]:
new_df.drop(['FireplaceQu'], axis=1, inplace=True)

In [ ]:
nan_feel_cols = nan_df[(nan_df['percentage'] < 45) & (nan_df['percentage'] > 0)]['column'].unique()

In [ ]:
obj_cols = new_df[nan_feel_cols].select_dtypes(include=['object']).columns
float_cols = new_df[nan_feel_cols].select_dtypes(include=['float']).columns

In [ ]:
new_df[obj_cols] = new_df[obj_cols].fillna('None')

In [ ]:
for col in float_cols:
    new_df[col] = new_df[col].fillna(new_df[col].mode().iloc[0])

In [ ]:
new_df.head()

Let's plot co-relation between labels and some features

In [ ]:
# All houses sold between 2006-2010, with significant drop in 2010
sns.countplot(new_df['YrSold']);

In [ ]:
# Most of the houses are built after 1950, with significant rise in 2000
sns.distplot(new_df['YearBuilt']);

In [ ]:
sns.distplot(new_df['LotFrontage']);

In [ ]:
sns.distplot(new_df['LotArea']);

In [ ]:
for obj in obj_cols:
    print(obj)
    print(new_df[obj].unique())

In [ ]:
new_df_1 = pd.get_dummies(new_df)

In [ ]:
new_df_1.head()

In [ ]:
train_df_1 = new_df_1[:train_idx]
test_df_1 = new_df_1[train_idx:]

# Let's try different regression models with GridsearchCV

Below parameters are set after grid search

In [ ]:
rf = RandomForestRegressor(max_depth=40, min_samples_leaf=3, min_samples_split=8,
                           n_estimators=5000, random_state=17)

In [ ]:
rf.fit(train_df_1, labels)

In [ ]:
lgbm = LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
               feature_fraction=0.4, learning_rate=0.01, max_bin=200,
               n_estimators=5000, num_leaves=8, objective='regression')

In [ ]:
lgbm.fit(train_df_1, labels, eval_metric='rmse')

In [ ]:
model = CatBoostRegressor(random_seed=17, depth=6, iterations=5000, learning_rate=0.01)

In [ ]:
model.fit(train_df_1, labels)

In [ ]:
xgb = XGBRegressor(booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', max_delta_step=0,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1, learning_rate=0.1,
             n_estimators=2500)

In [ ]:
xgb.fit(train_df_1, labels)

# Predict

In [ ]:
preds1 = rf.predict(test_df_1)

In [ ]:
pred2 = lgbm.predict(test_df_1)

In [ ]:
pred3 = model.predict(test_df_1)

In [ ]:
pred4 = xgb.predict(test_df_1)

In [ ]:
preds = (preds1 + pred2 + pred3 + pred4) / 4

In [ ]:
sub_csv = pd.DataFrame({
        "Id": test_df["Id"],
        "SalePrice": preds
    })

In [ ]:
sub_csv.to_csv('submission.csv', index=False)